In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chest-Cancer-Classification-MLFlow-DVC\\research'

In [3]:
# We need to be on the project root folder to ensure our code works with relative paths and avoid the FileNotFoundError.
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Chest-Cancer-Classification-MLFlow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# @dataclass(frozen=True): This decorator is used to automatically generate special methods like __init__(), __repr__(), __eq__(), etc., for the class. 
# The frozen=True parameter makes the instances of this class immutable, meaning that its fields cannot be modified after they are created.
# to ensure that the configuration settings remain constant throughout the execution of the program.
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    """
    ConfigurationManager handles the setup and retrieval of configuration settings for the pipeline.
    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Load configuration settings from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create necessary directories for artifacts
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Retrieve the configuration for preparing the base model.

        Returns:
            PrepareBaseModelConfig: An object containing the configuration settings for preparing the base model.
        """
        # Extract the base model preparation configuration from the loaded config
        config = self.config.prepare_base_model

        # Create directories specified in the base model configuration
        create_directories([config.root_dir])

        # Instantiate and return the PrepareBaseModelConfig object with the necessary settings
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    """
    PrepareBaseModel class is responsible for loading a pre-trained base model,
    updating it with additional layers, and saving the model to specified paths.
    """

    def __init__(self, config: PrepareBaseModelConfig):
        """
        Initialize the PrepareBaseModel with the given configuration.

        Args:
            config (PrepareBaseModelConfig): Configuration object for preparing the base model.
        """
        self.config = config

    def get_base_model(self):
        """
        Load the pre-trained ResNet50 base model with the specified parameters and save it.
        """
        # Load the pre-trained VGG16 model with specified parameters
        self.model = tf.keras.applications.ResNet50(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        # Save the loaded base model to the specified path
        self.save_model(path=self.config.base_model_path, model=self.model)

    # @staticmethod decorator is used for methods that do not need to access the instance attributes (self) or class variables.
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        """
        Prepare the full model by adding custom layers to the base model and compiling it.

        Args:
            model (tf.keras.Model): The base model to which custom layers will be added.
            classes (int): Number of output classes for the final layer.
            freeze_all (bool): If True, freeze all layers in the base model.
            freeze_till (int or None): If specified, freeze layers up to this index.
            learning_rate (float): Learning rate for the optimizer.

        Returns:
            tf.keras.Model: The full model with added custom layers.
        """
        # Freeze layers in the base model as specified
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        # Add custom layers to the base model
        flatten_in = tf.keras.layers.GlobalAveragePooling2D()(model.output)
        dropout = tf.keras.layers.Dropout(0.35)(flatten_in)  # Adding dropout to prevent overfitting
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(dropout)


        # Create the full model
        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)


        # Compile the full model with specified optimizer, loss function, and metrics
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),  # Changed optimizer to Adam
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        # Print the model summary
        full_model.summary()
        return full_model

    def update_base_model(self):
        """
        Update the base model by adding custom layers and compile it.
        Save the updated model to the specified path.
        """
        # Prepare the full model with custom layers and specified parameters
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till=35,  # Fine-tune the last 35 layers
            learning_rate=self.config.params_learning_rate
        )

        # Save the updated full model to the specified path
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Save the given model to the specified path.

        Args:
            path (Path): Path to save the model.
            model (tf.keras.Model): The model to be saved.
        """
        model.save(path)

In [10]:
try:
    # Initialize the ConfigurationManager to load configuration and parameter settings
    config = ConfigurationManager()
    
    # Retrieve the configuration settings for preparing the base model
    prepare_base_model_config = config.get_prepare_base_model_config()
    
    # Initialize the PrepareBaseModel process with the configuration settings
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    
    # Download the pre-trained base model (e.g., VGG16) with specified configurations
    prepare_base_model.get_base_model()
    
    # Update the base model by adding custom layers and compiling it with specified parameters
    prepare_base_model.update_base_model()
    
except Exception as e:
    # Log the exception details and raise it
    logger.error(f"An error occurred: {e}")
    raise e


[2024-07-31 20:47:14,526: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-07-31 20:47:14,539: INFO: common: YAML file: params.yaml loaded successfully]
[2024-07-31 20:47:14,545: INFO: common: Created directory at: artifacts]
[2024-07-31 20:47:14,552: INFO: common: Created directory at: artifacts/prepare_base_model]
[2024-07-31 20:47:20,768: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                